In [1]:
import pandas as pd 
import numpy as np
import datetime
from pathlib import Path
import os

pd.set_option('display.max_columns', 500)


In [2]:
data_path = Path(os.getcwd()) / 'data' / 'WEC2022_Data'

In [3]:
data_path

WindowsPath('c:/wec_2022/wec_2022/data/WEC2022_Data')

In [4]:
bkg = pd.read_csv(data_path / 'BKG_train.csv', sep=';')

In [5]:
tkt = pd.read_csv(data_path / 'TKT_train.csv', sep=';')

In [6]:
fcp = pd.read_csv(data_path / 'FCP_train.csv', sep=';')

In [7]:
emd = pd.read_csv(data_path / 'EMD_train.csv', sep=';')

In [8]:
tkt

,BOOKING_ID,TICKET_NUMBER,ORIGINAL_TICKET_NUMBER,FORM_OF_PAYMENT,CURRENCY,TOTAL_PRICE,TOTAL_PRICE_PLN,PAX_GENDER,PAX_TYPE,CORPORATE_CONTRACT_FLG,LOYAL_CUSTOMER,LOYAL_CUSTOMER_ID,LOYAL_CUSTOMER_DATE_OF_BIRTH,LOYAL_CUSTOMER_REGISTERED_DATE
0,4443948267837742,2433160688245721,9379265972706784,CCCA5,EUR,564.14,2350.94,M,ADULT,N,N,NaN,NaN,NaN
1,8827981391920441,2285471225231829,9379265972706784,CA,PLN,470.50,470.50,M,ADULT,N,N,NaN,NaN,NaN
2,7756899638586197,8700163489079041,9379265972706784,CA,PLN,218.49,218.49,M,ADULT,N,N,NaN,NaN,NaN
3,6411974034589912,1412002991872234,9379265972706784,CA,PLN,720.61,720.61,M,ADULT,N,N,NaN,NaN,NaN
4,6730427035328795,1715275704997737,9379265972706784,CCCA5,PLN,415.61,415.61,M,ADULT,N,N,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3944498,7532867952732900,7387270018960696,9379265972706784,CCCA5,SEK,2395.00,998.24,M,ADULT,N,Y,4.904725e+15,1968-12-07,2003-05-07
3944499,4390923384627090,4028422919052082,9379265972706784,CA,EUR,109.17,467.83,M,ADULT,N,N,NaN,NaN,NaN
3944500,9268232053474716,6586299214365763,9379265972706784,CCVI4,USD,532.90,2007.33,M,ADULT,N,N,NaN,NaN,NaN
3944501,451171036064944,233248134999409,9379265972706784,CCTP1,USD,692.50,2602.97,M,ADULT,N,N,NaN,NaN,NaN


In [9]:
stay_lenght_map = {-9999: None}

In [10]:
bkg['STAY_LENGTH_D'] = bkg['STAY_LENGTH_D'].replace(stay_lenght_map)
bkg['BOOKING_ARRIVAL_TIME_UTC'] = pd.to_datetime(bkg['BOOKING_ARRIVAL_TIME_UTC'])
bkg['BOOKING_DEPARTURE_TIME_UTC'] = pd.to_datetime(bkg['BOOKING_DEPARTURE_TIME_UTC'])

In [11]:
bkg['flight_len'] = bkg['BOOKING_ARRIVAL_TIME_UTC'] - bkg['BOOKING_DEPARTURE_TIME_UTC']
bkg['flight_len'] = bkg['flight_len'].apply(lambda x: x.seconds / 3600)

In [12]:
bkg

,BOOKING_ID,SALES_DATE,SALES_MARKET,SALES_CHANNEL,TRIP_TYPE,BOOKING_WINDOW_D,STAY_LENGTH_D,BOOKING_LONG_HOUL_FLAG,BOOKING_DOMESTIC_FLAG,FLIGHT_COUPONS,SEGMENTS,PAX_N,INTINERARY,BOOKING_ORIGIN_AIRPORT,BOOKING_ORIGIN_COUNTRY_CODE,BOOKING_DEPARTURE_TIME_UTC,BOOKING_DESTINATION_AIRPORT,BOOKING_DESTINATION_COUNTRY_CODE,BOOKING_ARRIVAL_TIME_UTC,UPGRADED_FLAG,UPGRADE_SALES_DATE,UPGRADE_TYPE,flight_len
0,2754285175826930,2007-01-26,3823004278339643,AGENTS,ROUND TRIP,61.0,9.0,N,N,2,2,1,LFF-GMW-LFF,LFF,EE,2007-03-28 04:00:00,GMW,DE,2007-04-06 21:10:00,N,NaN,NaN,17.166667
1,667762693252160,2007-10-05,6982998843464221,AGENTS,ROUND TRIP,10.0,4.0,Y,N,4,2,1,VMY-HIX-OUO-OIH-OUO-HIX-VMY,HIX,US,2007-10-15 22:25:00,OIH,PL,2007-10-19 00:35:00,N,NaN,NaN,2.166667
2,7185592089077946,2007-01-08,3823004278339643,AGENTS,ONE WAY,38.0,None,N,N,1,1,1,WXA-OUO,WXA,FR,2007-02-15 06:30:00,OUO,PL,2007-02-15 08:45:00,N,NaN,NaN,2.250000
3,4685868825417887,2007-11-14,2460355646543430,AGENTS,ONE WAY,16.0,None,N,N,1,1,1,UGK-OUO,UGK,NL,2007-11-30 18:55:00,OUO,PL,2007-11-30 20:55:00,N,NaN,NaN,2.000000
4,6948280039050277,2007-09-26,4802659914165873,AGENTS,ROUND TRIP,29.0,4.0,N,N,8,2,2,EDN-OUO-FOH-OUO-EDN,EDN,MD,2007-10-25 14:55:00,FOH,UA,2007-10-29 12:05:00,N,NaN,NaN,21.166667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2795935,8277851166644641,2007-10-08,4391133056062766,LOT.COM,ONE WAY,6.0,None,N,N,4,1,2,FWU-OUO-KNH,FWU,CY,2007-10-14 00:40:00,KNH,RU,2007-10-14 10:50:00,N,NaN,NaN,10.166667
2795936,6336772439230224,2007-04-26,8557423498357146,LOT.COM,ROUND TRIP,19.0,1.0,N,N,4,2,1,JHS-OUO-FBI-OUO-JHS,JHS,PL,2007-05-15 12:55:00,FBI,GB,2007-05-16 21:50:00,N,NaN,NaN,8.916667
2795937,2491938906106596,2007-01-08,6745023307045216,AGENTS,ROUND TRIP,10.0,3.0,N,N,2,2,1,FBI-OUO-FBI,FBI,GB,2007-01-18 10:25:00,OUO,PL,2007-01-21 17:20:00,N,NaN,NaN,6.916667
2795938,4105882171748456,2007-07-16,4802659914165873,AGENTS,MULTICITY,14.0,1.0,N,N,2,2,1,FBI-OUO-ELO,OUO,PL,2007-07-30 08:45:00,NaN,NaN,2007-07-31 08:00:00,N,NaN,NaN,23.250000


In [13]:
bkg_drop_cols = ['UPGRADED_FLAG', 'UPGRADE_TYPE', 'UPGRADE_SALES_DATE']
bkg = bkg.drop(columns=bkg_drop_cols)

In [14]:
bkg

,BOOKING_ID,SALES_DATE,SALES_MARKET,SALES_CHANNEL,TRIP_TYPE,BOOKING_WINDOW_D,STAY_LENGTH_D,BOOKING_LONG_HOUL_FLAG,BOOKING_DOMESTIC_FLAG,FLIGHT_COUPONS,SEGMENTS,PAX_N,INTINERARY,BOOKING_ORIGIN_AIRPORT,BOOKING_ORIGIN_COUNTRY_CODE,BOOKING_DEPARTURE_TIME_UTC,BOOKING_DESTINATION_AIRPORT,BOOKING_DESTINATION_COUNTRY_CODE,BOOKING_ARRIVAL_TIME_UTC,flight_len
0,2754285175826930,2007-01-26,3823004278339643,AGENTS,ROUND TRIP,61.0,9.0,N,N,2,2,1,LFF-GMW-LFF,LFF,EE,2007-03-28 04:00:00,GMW,DE,2007-04-06 21:10:00,17.166667
1,667762693252160,2007-10-05,6982998843464221,AGENTS,ROUND TRIP,10.0,4.0,Y,N,4,2,1,VMY-HIX-OUO-OIH-OUO-HIX-VMY,HIX,US,2007-10-15 22:25:00,OIH,PL,2007-10-19 00:35:00,2.166667
2,7185592089077946,2007-01-08,3823004278339643,AGENTS,ONE WAY,38.0,None,N,N,1,1,1,WXA-OUO,WXA,FR,2007-02-15 06:30:00,OUO,PL,2007-02-15 08:45:00,2.250000
3,4685868825417887,2007-11-14,2460355646543430,AGENTS,ONE WAY,16.0,None,N,N,1,1,1,UGK-OUO,UGK,NL,2007-11-30 18:55:00,OUO,PL,2007-11-30 20:55:00,2.000000
4,6948280039050277,2007-09-26,4802659914165873,AGENTS,ROUND TRIP,29.0,4.0,N,N,8,2,2,EDN-OUO-FOH-OUO-EDN,EDN,MD,2007-10-25 14:55:00,FOH,UA,2007-10-29 12:05:00,21.166667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2795935,8277851166644641,2007-10-08,4391133056062766,LOT.COM,ONE WAY,6.0,None,N,N,4,1,2,FWU-OUO-KNH,FWU,CY,2007-10-14 00:40:00,KNH,RU,2007-10-14 10:50:00,10.166667
2795936,6336772439230224,2007-04-26,8557423498357146,LOT.COM,ROUND TRIP,19.0,1.0,N,N,4,2,1,JHS-OUO-FBI-OUO-JHS,JHS,PL,2007-05-15 12:55:00,FBI,GB,2007-05-16 21:50:00,8.916667
2795937,2491938906106596,2007-01-08,6745023307045216,AGENTS,ROUND TRIP,10.0,3.0,N,N,2,2,1,FBI-OUO-FBI,FBI,GB,2007-01-18 10:25:00,OUO,PL,2007-01-21 17:20:00,6.916667
2795938,4105882171748456,2007-07-16,4802659914165873,AGENTS,MULTICITY,14.0,1.0,N,N,2,2,1,FBI-OUO-ELO,OUO,PL,2007-07-30 08:45:00,NaN,NaN,2007-07-31 08:00:00,23.250000


In [15]:
tkt

,BOOKING_ID,TICKET_NUMBER,ORIGINAL_TICKET_NUMBER,FORM_OF_PAYMENT,CURRENCY,TOTAL_PRICE,TOTAL_PRICE_PLN,PAX_GENDER,PAX_TYPE,CORPORATE_CONTRACT_FLG,LOYAL_CUSTOMER,LOYAL_CUSTOMER_ID,LOYAL_CUSTOMER_DATE_OF_BIRTH,LOYAL_CUSTOMER_REGISTERED_DATE
0,4443948267837742,2433160688245721,9379265972706784,CCCA5,EUR,564.14,2350.94,M,ADULT,N,N,NaN,NaN,NaN
1,8827981391920441,2285471225231829,9379265972706784,CA,PLN,470.50,470.50,M,ADULT,N,N,NaN,NaN,NaN
2,7756899638586197,8700163489079041,9379265972706784,CA,PLN,218.49,218.49,M,ADULT,N,N,NaN,NaN,NaN
3,6411974034589912,1412002991872234,9379265972706784,CA,PLN,720.61,720.61,M,ADULT,N,N,NaN,NaN,NaN
4,6730427035328795,1715275704997737,9379265972706784,CCCA5,PLN,415.61,415.61,M,ADULT,N,N,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3944498,7532867952732900,7387270018960696,9379265972706784,CCCA5,SEK,2395.00,998.24,M,ADULT,N,Y,4.904725e+15,1968-12-07,2003-05-07
3944499,4390923384627090,4028422919052082,9379265972706784,CA,EUR,109.17,467.83,M,ADULT,N,N,NaN,NaN,NaN
3944500,9268232053474716,6586299214365763,9379265972706784,CCVI4,USD,532.90,2007.33,M,ADULT,N,N,NaN,NaN,NaN
3944501,451171036064944,233248134999409,9379265972706784,CCTP1,USD,692.50,2602.97,M,ADULT,N,N,NaN,NaN,NaN


In [16]:
bkg

,BOOKING_ID,SALES_DATE,SALES_MARKET,SALES_CHANNEL,TRIP_TYPE,BOOKING_WINDOW_D,STAY_LENGTH_D,BOOKING_LONG_HOUL_FLAG,BOOKING_DOMESTIC_FLAG,FLIGHT_COUPONS,SEGMENTS,PAX_N,INTINERARY,BOOKING_ORIGIN_AIRPORT,BOOKING_ORIGIN_COUNTRY_CODE,BOOKING_DEPARTURE_TIME_UTC,BOOKING_DESTINATION_AIRPORT,BOOKING_DESTINATION_COUNTRY_CODE,BOOKING_ARRIVAL_TIME_UTC,flight_len
0,2754285175826930,2007-01-26,3823004278339643,AGENTS,ROUND TRIP,61.0,9.0,N,N,2,2,1,LFF-GMW-LFF,LFF,EE,2007-03-28 04:00:00,GMW,DE,2007-04-06 21:10:00,17.166667
1,667762693252160,2007-10-05,6982998843464221,AGENTS,ROUND TRIP,10.0,4.0,Y,N,4,2,1,VMY-HIX-OUO-OIH-OUO-HIX-VMY,HIX,US,2007-10-15 22:25:00,OIH,PL,2007-10-19 00:35:00,2.166667
2,7185592089077946,2007-01-08,3823004278339643,AGENTS,ONE WAY,38.0,None,N,N,1,1,1,WXA-OUO,WXA,FR,2007-02-15 06:30:00,OUO,PL,2007-02-15 08:45:00,2.250000
3,4685868825417887,2007-11-14,2460355646543430,AGENTS,ONE WAY,16.0,None,N,N,1,1,1,UGK-OUO,UGK,NL,2007-11-30 18:55:00,OUO,PL,2007-11-30 20:55:00,2.000000
4,6948280039050277,2007-09-26,4802659914165873,AGENTS,ROUND TRIP,29.0,4.0,N,N,8,2,2,EDN-OUO-FOH-OUO-EDN,EDN,MD,2007-10-25 14:55:00,FOH,UA,2007-10-29 12:05:00,21.166667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2795935,8277851166644641,2007-10-08,4391133056062766,LOT.COM,ONE WAY,6.0,None,N,N,4,1,2,FWU-OUO-KNH,FWU,CY,2007-10-14 00:40:00,KNH,RU,2007-10-14 10:50:00,10.166667
2795936,6336772439230224,2007-04-26,8557423498357146,LOT.COM,ROUND TRIP,19.0,1.0,N,N,4,2,1,JHS-OUO-FBI-OUO-JHS,JHS,PL,2007-05-15 12:55:00,FBI,GB,2007-05-16 21:50:00,8.916667
2795937,2491938906106596,2007-01-08,6745023307045216,AGENTS,ROUND TRIP,10.0,3.0,N,N,2,2,1,FBI-OUO-FBI,FBI,GB,2007-01-18 10:25:00,OUO,PL,2007-01-21 17:20:00,6.916667
2795938,4105882171748456,2007-07-16,4802659914165873,AGENTS,MULTICITY,14.0,1.0,N,N,2,2,1,FBI-OUO-ELO,OUO,PL,2007-07-30 08:45:00,NaN,NaN,2007-07-31 08:00:00,23.250000


In [23]:
df = fcp.merge(tkt, on='TICKET_NUMBER', how='left')
df.shape

(9224587, 31)

In [24]:
df = df.merge(bkg, on='BOOKING_ID', how='left')
df.shape

(9224587, 50)

In [19]:
# df = df.merge(emd, how='left', left_on='TICKET_NUMBER', right_on='REFERENCE_TICKET_NUMBER')
# df.shape

(10998146, 64)

In [26]:
df.shape

(9224587, 50)

In [32]:
df.loc[:, [col for col in df.columns if 'DATE' in col]]

,FLIGHT_DATE_LOCAL,UPGRADE_SALES_DATE,LOYAL_CUSTOMER_DATE_OF_BIRTH,LOYAL_CUSTOMER_REGISTERED_DATE,SALES_DATE
0,2007-05-16,NaN,NaN,NaN,2007-01-11
1,2007-08-26,NaN,NaN,NaN,2007-01-10
2,2007-03-20,NaN,NaN,NaN,2007-01-10
3,2007-04-08,NaN,NaN,NaN,2007-03-27
4,2007-01-17,NaN,NaN,NaN,2007-01-10
...,...,...,...,...,...
9224582,2007-02-05,NaN,NaN,NaN,2007-02-05
9224583,2007-08-04,NaN,NaN,NaN,2007-04-02
9224584,2007-06-02,NaN,NaN,NaN,2007-04-27
9224585,2007-09-16,NaN,NaN,NaN,2007-08-10


In [83]:
(df[df['if_additional_upgrade'] == 1]['ORIGIN_AIRPORT_CODE'].value_counts() / df['ORIGIN_AIRPORT_CODE'].value_counts()).sort_values(ascending=False).head(30)

FDB    1.000000
ALY    1.000000
JUB    1.000000
VMI    1.000000
EKT    1.000000
KGI    1.000000
BRU    1.000000
SUQ    1.000000
FJV    0.666667
SDU    0.666667
EHC    0.666667
DOE    0.666667
EHU    0.615385
UXF    0.500000
FDI    0.500000
ARZ    0.500000
FWA    0.454545
AIV    0.400000
FAV    0.400000
MNY    0.368421
UTW    0.352941
KLL    0.333333
XUF    0.333333
WHK    0.333333
GXW    0.333333
GDU    0.324374
GFD    0.300000
VTZ    0.280000
YME    0.267974
SUX    0.263609
Name: ORIGIN_AIRPORT_CODE, dtype: float64

In [75]:
df[df['if_additional_upgrade'] == 0]['ORIGIN_AIRPORT_CODE'].value_counts().head(20)

OUO    3468249
LFF     319750
EIE     153483
GMW     128093
UGK     123049
UIT     119849
VMX     119020
NTH     116130
FBI     114731
LFN     114059
VIM     112990
WXA     109156
HIX     101107
XMK     100981
WJB     100393
CYE      94954
OIH      85682
AXT      82947
JIA      81455
YIU      78652
Name: ORIGIN_AIRPORT_CODE, dtype: int64

In [87]:
(df['MARKETING_CARRIER'] - df['OPERATIONAL_CARRIER'])

 0                   9015266
-2969159038110937      45585
-2241741366190995      35092
-5060948346632247      30931
-6370019050174410      30816
                      ...   
 461153743123512           1
-1963270163453875          1
-1864415057312114          1
-5158226703011825          1
-3216852323409272          1
Length: 74, dtype: int64

# Co zmieniamy 
1. 

In [ ]:
drop_cols = ['TICKET_NUMBER', 'UPGRADE_SALES_DATE', ]
oh_cols = ['ORIGIN_AIRPORT_CODE', 'DESTINATION_AIRPORT_CODE', 'SALES_DATE', 'FLIGHT_DATE_LOCAL', ]

In [35]:
df['FLIGHT_DATE_LOCAL'] = pd.to_datetime(df['FLIGHT_DATE_LOCAL'])
df['SALES_DATE'] = pd.to_datetime(df['SALES_DATE'])
df['sale_to_flight_time'] = df['FLIGHT_DATE_LOCAL'] - df['SALES_DATE']
df['sale_to_flight_time'] = df['sale_to_flight_time'].apply(lambda x: x.seconds / 3600)

In [44]:
df['TIME_DEPARTURE_LOCAL_TIME'] = pd.to_datetime(df['TIME_DEPARTURE_LOCAL_TIME'])
df['TIME_DEPARTURE_LOCAL_TIME'] = df['TIME_DEPARTURE_LOCAL_TIME'].apply(lambda x: x.hour)

In [67]:
df['if_additional_upgrade'] = np.where(np.isin(df['TICKET_NUMBER'], emd['REFERENCE_TICKET_NUMBER'].unique()), 1, 0)

In [68]:
df[df['if_additional_upgrade'] == 1]

,TICKET_NUMBER,COUPON_NUMBER,ORIGIN_AIRPORT_CODE,DESTINATION_AIRPORT_CODE,FLIGHT_DATE_LOCAL,TIME_DEPARTURE_LOCAL_TIME,FLIGHT_DISTANCE,FLIGHT_RANGE,MARKETING_CARRIER,OPERATIONAL_CARRIER,BOOKED_CLASS,BOOKED_CABIN,AIRCRAFT_TYPE,FARE_BASIS,VAB,UPGRADED_FLAG,UPGRADE_TYPE,UPGRADE_SALES_DATE,BOOKING_ID,ORIGINAL_TICKET_NUMBER,FORM_OF_PAYMENT,CURRENCY,TOTAL_PRICE,TOTAL_PRICE_PLN,PAX_GENDER,PAX_TYPE,CORPORATE_CONTRACT_FLG,LOYAL_CUSTOMER,LOYAL_CUSTOMER_ID,LOYAL_CUSTOMER_DATE_OF_BIRTH,LOYAL_CUSTOMER_REGISTERED_DATE,SALES_DATE,SALES_MARKET,SALES_CHANNEL,TRIP_TYPE,BOOKING_WINDOW_D,STAY_LENGTH_D,BOOKING_LONG_HOUL_FLAG,BOOKING_DOMESTIC_FLAG,FLIGHT_COUPONS,SEGMENTS,PAX_N,INTINERARY,BOOKING_ORIGIN_AIRPORT,BOOKING_ORIGIN_COUNTRY_CODE,BOOKING_DEPARTURE_TIME_UTC,BOOKING_DESTINATION_AIRPORT,BOOKING_DESTINATION_COUNTRY_CODE,BOOKING_ARRIVAL_TIME_UTC,flight_len,sale_to_flight_time,if_additional_upgrade
2,2822510487722780,2,OUO,HIX,2007-03-20,16.0,7521,LONG-HAUL,2434615205489683,2434615205489683,U,Economy,788,ULPRO11,BASIC,N,NaN,NaN,6366425806050664,9379265972706784,CCVI4,USD,648.77,2270.18,F,ADULT,N,N,NaN,NaN,NaN,2007-01-10,6982998843464221,LOT.COM,ROUND TRIP,55.0,15.0,Y,N,2,2,1,HIX-OUO-HIX,HIX,US,2007-03-06 03:35:00,OUO,PL,2007-03-21 01:50:00,22.250000,0.0,1
10,325528216485266,3,UGK,OUO,2007-01-18,19.0,1102,SHORT-HAUL,2434615205489683,2434615205489683,P,Premium,73H,PSFXP0,SEMI-FLEX,N,NaN,NaN,7500287936823074,9379265972706784,CCCA5,PLN,2889.55,2889.55,M,ADULT,N,Y,4.891858e+15,1979-09-12,2006-10-22,2007-01-11,2460355646543430,LOT.COM,ROUND TRIP,3.0,4.0,N,N,4,2,1,EIE-OUO-UGK-OUO-EIE,EIE,PL,2007-01-14 17:05:00,UGK,NL,2007-01-18 22:40:00,5.583333,0.0,1
13,4065421878013652,1,OUO,ZNT,2007-05-10,22.0,2237,SHORT-HAUL,2434615205489683,2434615205489683,L,Economy,738,LSAVM10,SAVER,N,NaN,NaN,4463993914697325,9379265972706784,CCCA5,PLN,806.01,806.01,M,ADULT,N,N,NaN,NaN,NaN,2007-01-11,2460355646543430,LOT.COM,ROUND TRIP,119.0,5.0,N,N,2,2,1,OUO-ZNT-OUO,OUO,PL,2007-05-10 20:20:00,ZNT,AM,2007-05-15 04:30:00,8.166667,0.0,1
14,7700133142957541,4,OUO,OIH,2007-04-01,7.0,299,DOMESTIC,2434615205489683,2434615205489683,O,Economy,E70,OZSAVJ0,SAVER,N,NaN,NaN,1237136580053375,9379265972706784,CCCA5,PLN,449.55,449.55,M,ADULT,N,Y,1.437026e+15,1943-11-01,1988-05-27,2007-01-11,2460355646543430,LOT.COM,ROUND TRIP,73.0,7.0,N,N,8,2,2,OIH-OUO-FWU-OUO-OIH,OIH,PL,2007-03-25 10:00:00,FWU,CY,2007-04-01 06:30:00,20.500000,0.0,1
19,1814434208957439,1,LPF,OUO,2007-01-19,11.0,523,SHORT-HAUL,2434615205489683,2434615205489683,T,Economy,DH4,TSTDK0,STANDARD,N,NaN,NaN,5137965917006737,9379265972706784,CCCA5,EUR,228.20,953.40,M,ADULT,N,Y,4.381709e+15,1960-06-03,1990-04-27,2007-01-10,5492600422950699,LOT.COM,ROUND TRIP,9.0,3.0,N,N,2,2,1,LPF-OUO-LPF,LPF,DE,2007-01-19 10:30:00,OUO,PL,2007-01-22 17:50:00,7.333333,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9224525,7106916144504638,2,OUO,CYE,2007-10-30,16.0,6863,LONG-HAUL,2434615205489683,2434615205489683,L,Economy,788,LLATAN,FLEX,N,NaN,NaN,6299931923522541,9379265972706784,CCCA5,UAH,16388.00,2127.16,M,ADULT,N,N,NaN,NaN,NaN,2007-09-22,1319526191137192,LOT.COM,ROUND TRIP,38.0,11.0,Y,N,4,2,1,BIE-OUO-CYE-OUO-BIE,BIE,UA,2007-10-30 12:30:00,CYE,US,2007-11-10 15:20:00,2.833333,0.0,1
9224532,5727356101030207,2,OUO,SIB,2007-09-03,17.0,1038,SHORT-HAUL,2434615205489683,2434615205489683,T,Economy,E75,T1TAN,OTHER,N,NaN,NaN,4135817243835119,9379265972706784,CCVI4,CHF,513.00,1830.44,M,ADULT,N,N,NaN,NaN,NaN,2007-04-06,7445666648510309,LOT.COM,ONE WAY,150.0,None,Y,N,6,1,3,JZE-OUO-SIB,JZE,CN,2007-09-03 00:45:00,SIB,CH,2007-09-03 17:05:00,16.333333,0.0,1
9224545,3683302737705684,2,OUO,CYE,2007-11-02,16.0,6863,LONG-HAUL,2434615205489683,2434615205489683,L,Economy,789,LLWTAN,FLEX,N,NaN,NaN,872614284487879,9379265972706784,CCCA5,CAD,774.60,2178.33,M,ADULT,N,N,NaN,NaN,NaN,2007-09-27,6165704389606935,AGENT

In [70]:
emd

,EMD_TICKET_NUMBER,COUPON_NUMBER,SALES_DATE,SALES_CHANNEL,REFERENCE_TICKET_NUMBER,ORIGIN_AIRPORT_CODE,DESTINATION_AIRPORT_CODE,COD_RFISC,SUBCATEGORY,CATEGORY,GROUP,CURRENCY,TOTAL_PRICE,TOTAL_PRICE_PLN
0,4252771510645404,2,2007-02-15,AGENTS,2797844356918533,OUO,NTH,0GO,ADDITIONAL BAGGAGE,BAGGAGE,ANCILLARY,HUF,35000.0,467.57
1,4252771510645404,3,2007-02-15,AGENTS,2797844356918533,NTH,OUO,0GO,ADDITIONAL BAGGAGE,BAGGAGE,ANCILLARY,HUF,35000.0,467.57
2,4252771510645404,1,2007-02-15,AGENTS,2797844356918533,VMX,OUO,0GO,ADDITIONAL BAGGAGE,BAGGAGE,ANCILLARY,HUF,35000.0,467.57
3,4252771510645404,4,2007-02-15,AGENTS,2797844356918533,OUO,VMX,0GO,ADDITIONAL BAGGAGE,BAGGAGE,ANCILLARY,HUF,35000.0,467.57
4,7420318561824616,3,2007-02-15,AGENTS,2535201643451801,NTH,OUO,0GO,ADDITIONAL BAGGAGE,BAGGAGE,ANCILLARY,HUF,35000.0,467.57
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1135122,911632049382988,1,2007-11-16,ATO_CTO,3185499926528786,LFF,VIM,EXS,EXTRA SEAT,SEAT SELECTION,ANCILLARY,EUR,346.0,1486.76
1135123,911632049382988,2,2007-11-16,ATO_CTO,3185499926528786,VIM,LFF,EXS,EXTRA SEAT,SEAT SELECTION,ANCILLARY,EUR,346.0,1486.76
1135124,2763840285725474,1,2007-01-18,ATO_CTO,8099945440216893,NaN,NaN,0LA,VOUCHER SERVICES,OTHER,TECHNICAL,EUR,56.0,233.74
1135125,9787387898539181,1,2007-01-18,ATO_CTO,6572767204990065,NaN,NaN,0LA,VOUCHER SERVICES,OTHER,TECHNICAL,EUR,56.0,233.74
